### config.py contains the client id and app secret. Ensure the values in this file are correct
### praw is just a reddit api wrapper for python

In [ ]:
import config
import requests
import praw

In [ ]:
user_agent = "python:ministryofedCSC_redditscraper:v0.0.1 (by rahm)"

In [ ]:
reddit = praw.Reddit(client_id=config.client_id,
                     client_secret=config.client_secret,
                     user_agent=user_agent,
                    )

In [ ]:
print(reddit.read_only)

### get subreddits to search can be done here. for now just making a short list for demo purposes

In [ ]:
subredditsToSearch = ['ontario', 'education']

In [ ]:
# TODO
# this function is an attempt to get around the 1000 limit
def getNewSubmissions(subToSearch, limit):
    if limit > 100:
        # max limit is 100, so if larger, set a throttled limit and then save the difference
        thrLimit = 100
        limit = limit - 100
    else:
        thrLim = limit
    submissions = reddit.subreddit(subToSearch).new(limit=thrLim)
    for i in range(len(submissions)):
        sub = submissions[i]
        print(sub.title)
        yield sub
        # if this is the last iteration of the loop, we save the id of this last submission
        # we can use this last submission id as a starting point for our next batch of submissions
        if i == len(submissions):
            pass

## getting 1000 newest submissions for subreddits

In [ ]:
def getThousandSubmissions(subreddit):
    submissions = reddit.subreddit(subreddit).top(limit=1000)
    return submissions

In [ ]:
for subreddit in subredditsToSearch:
    submissions = getThousandSubmissions(subreddit)
    print(subreddit)
    print("=======================")
    i = 0
    for s in submissions:
        print(f"{i}.", s.title)
        # truncating a bit
        if i == 50:
            break
        i += 1

## get the comments from a submission

In [ ]:
# call this on a subreddit and it will continuously call replace_more
def flattenComments(submission):
    x = submission.comments
    while True:
        try:
            x.replace_more(limit=None)
            break
        except PossibleExceptions:
            print('Handling replace_more exception')
            sleep(1)
    return x.list()

In [ ]:
submissions = getThousandSubmissions('ontario')
for s in submissions:
    print(f"Submission title: {s.title}")
    # just want to test this on one submission
    flattenComments(s)
    allComments = s.comments.list()
    #print(allComments)
    for i in range(len(allComments)):
        comment = allComments[i]
        if isinstance(comment, praw.models.MoreComments):
            continue
            #comment.replace_more(limit=None)
        print("==========================================")
        print(f"{comment.author} said: {comment.body}")
        #truncating a bit
        if i == 50:
            break
    # only want to print 1 submission's comments for now
    break